In [1]:
import tensorflow 
from tensorflow import keras 
%load_ext tensorboard
import datetime
import keras_tuner
import numpy as np
import pandas as pd
import os
import PIL

In [2]:
os.chdir("D:\\Data Science\\Deep Learning\\POC project\\Covid19-dataset")

In [3]:
train_normal_images = os.listdir("train\\Normal")
train_covid_images = os.listdir("train\\Covid")
train_pneumonia_images = os.listdir("train\\Viral Pneumonia")

In [4]:
print(len(train_normal_images),len(train_covid_images),len(train_pneumonia_images))

70 111 70


In [5]:
img = PIL.Image.open("train\\Normal\\" + train_normal_images[0])
width, height = img.size
print(width,height)

1944 1708


### Width array and height array

In [6]:
width_array = np.array([])
height_array = np.array([])

In [7]:
#covid_images
for image in train_covid_images:
    img = PIL.Image.open("train\\Covid\\" + image)
    width, height = img.size
    width_array = np.append(width_array,width)
    height_array = np.append(height_array,height)    

In [8]:
#Normal_images
for image in train_normal_images:
    img = PIL.Image.open("train\\Normal\\" + image)
    width, height = img.size
    width_array = np.append(width_array,width)
    height_array = np.append(height_array,height)    

In [9]:
#Pneumonia_images
for image in train_pneumonia_images:
    img = PIL.Image.open("train\\Viral Pneumonia\\" + image)
    width, height = img.size
    width_array = np.append(width_array,width)
    height_array = np.append(height_array,height)   

In [10]:
print("The average width pixels are ",np.average(width_array))
print("The average height pixels are ", np.average(height_array))

The average width pixels are  1717.9721115537848
The average height pixels are  1439.98406374502


### Image Preprocessing

In [11]:
from keras.preprocessing.image import ImageDataGenerator

In [12]:
training = ImageDataGenerator("rescale" == 1/255.0,
                           rotation_range =40,
                           width_shift_range = 0.2,
                           height_shift_range = 0.2,
                           shear_range = 0.2,
                           zoom_range = 0.2,
                           horizontal_flip =True,
                           fill_mode = "nearest")
validation = ImageDataGenerator("rescale" == 1/255.0)

In [13]:
training_data = training.flow_from_directory('''D:\\Data Science\\Deep Learning\\POC project\\Covid19-dataset\\train'''
                                            ,target_size = (1718,1440)
                                            ,batch_size = 32
                                            ,color_mode = "grayscale"
                                            ,shuffle = True
                                            ,seed = 42
                                            ,class_mode = "categorical")

validation_data = validation.flow_from_directory('''D:\\Data Science\\Deep Learning\\POC project\\Covid19-dataset\\test'''
                                                ,target_size = (1718,1440)
                                                ,color_mode = "grayscale"
                                                ,batch_size = 22
                                                ,shuffle = False
                                                ,class_mode = None
                                                ,seed = 42)
                                                

Found 251 images belonging to 3 classes.
Found 66 images belonging to 3 classes.


In [14]:
x=np.concatenate([validation_data.next()[0] for i in range(validation_data.__len__())])
y=np.concatenate([validation_data.next()[1] for i in range(validation_data.__len__())])
print(x.shape)
print(y.shape)

(5154, 1440, 1)
(5154, 1440, 1)


## Building Neural Network

In [15]:
# def build_model(hp):
#     model = keras.Sequential([
# #         1st set
#         keras.layers.Conv2D(filters = hp.Int("filters_1",min_value = 4, max_value =12, step = 12),
#                            kernel_size = hp.Choice("kernel_size1",[3,4,5]),
#                            activation = "ReLU",
#                            strides = 1,
#                            padding = "same",
#                            input_shape = (1718,1440,1)),
#         keras.layers.MaxPooling2D((2,2)),
# #         2nd set
#         keras.layers.Conv2D(filters = hp.Int("filters_1",min_value = 4, max_value =12, step = 12),
#                            kernel_size = hp.Choice("filter_size2",[3,4,5]),
#                            activation = "ReLU",
#                            strides = 1,
#                            padding = "valid",
#                             ),
#         keras.layers.MaxPooling2D(hp.Choice("max_pooling1",[2,3,4])),
        
# #         Flattening and ANN part
#         keras.layers.Flatten(),
#         keras.layers.Dropout(hp.Choice("dropout_fraction",[0.05,0.2,0.3])),
#         keras.layers.Dense(units = hp.Int("num_neurons",min_value = 10, max_value= 32, step = 12),
#                           activation = "ReLU"),
#         keras.layers.Dense(units = 3, activation = "Softmax"),
        
#     ])
#     learning_rate = hp.Choice("learning_rate__",[1e-1,2e-1,3e-1,1e-2])
#     optimizers = hp.Choice("optimizers__",["Adam","Adagrad","Adamax","SGD"])
#     if optimizers == "Adam":
#         optimizer_chosen = keras.optimizers.Adam(learning_rate = learning_rate)
#     elif optimizers == "Adagrad":
#         optimizers_chosen = keras.optimizers.Adagrad(learning_rate = learning_rate)
#     elif optimizers == "Adamax":
#         optimizers_chosen = keras.optimizers.Adamax(learning_rate = learning_rate)
#     else:
#         optimizers_chosen = keras.optimizers.SGD(learning_rate = learning_rate)
        
#     model.compile(optimizer = optimizers,
#                  loss = 'categorical_crossentropy',
#                  metrics = "accuracy")
#     return model

In [16]:
# from keras_tuner import RandomSearch

In [17]:
# tuner  = RandomSearch(
#             build_model,
#             max_trials = 9,
#             objective = "val_accuracy",
#             project_name = "Chest_condition_classification",
#             directory = "D:\\Data Science\\Deep Learning\\POC project\\Covid19-dataset",
#             overwrite = True)

In [18]:
# tuner.search_space_summary()

In [19]:
# tuner.search(training_data,validation_data = (validation_data), batch_size = 64, 
#              epochs = 3)

In [23]:
model = keras.Sequential([
#         1st set
        keras.layers.Conv2D(filters = 32,
                           kernel_size = 5,
                           activation = "ReLU",
                           strides = 1,
                           padding = "valid",
                           input_shape = (1718,1440,1)),
        keras.layers.MaxPooling2D((4,4)),
#         2nd set
        keras.layers.Conv2D(filters = 25,
                           kernel_size = 5,
                           activation = "ReLU",
                           strides = 1,
                           padding = "valid",
                            ),
        keras.layers.MaxPooling2D(4,4),
        
#         Flattening and ANN part
        keras.layers.Flatten(),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(108,activation="ReLU"),
        keras.layers.Dense(units = 3, activation = "Softmax"),
        
    ])

model.compile(optimizer = keras.optimizers.Adam(learning_rate = 1e-1),
                 loss = 'categorical_crossentropy',
                 metrics = "accuracy")

In [24]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tensorflow.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [ ]:
# history = model.fit(training_data,validation_data = validation_data, batch_size = 1, epochs = 100,callbacks=[tensorboard_callback])

Epoch 1/100


In [1]:
from tensorflow.keras.models import load_model

In [3]:
new_model = load_model("C:\\Users\\hp\\OneDrive\\Desktop\\chest_condition_classification_model.h5")

In [4]:
new_model

In [5]:
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 1714, 1436, 10)    260       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 428, 359, 10)     0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 424, 355, 10)      2510      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 106, 88, 10)      0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 104, 86, 4)        364       
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 52, 43, 4)       

In [7]:
new_model.get_weights()

[array([[[[-0.56712466, -0.79014707, -0.5097945 , -0.58132863,
           -0.5231147 , -0.61026984, -0.5626419 , -0.67032087,
           -0.60024685, -0.34705546]],
 
         [[-0.6705811 , -0.7746191 , -0.6184159 , -0.57037103,
           -0.77694786, -0.7718185 , -0.61118585, -0.45729703,
           -0.46375677, -0.5585577 ]],
 
         [[-0.7058277 , -0.7738929 , -0.784575  , -0.51733094,
           -0.68804944, -0.59980243, -0.7789271 , -0.48307216,
           -0.546552  , -0.34576133]],
 
         [[-0.7150206 , -0.5855861 , -0.69395745, -0.5246772 ,
           -0.5341925 , -0.74872655, -0.64346826, -0.51822674,
           -0.4441406 , -0.5162497 ]],
 
         [[-0.5453437 , -0.7103116 , -0.7504667 , -0.34423047,
           -0.7806311 , -0.5837252 , -0.72610795, -0.5530005 ,
           -0.56934834, -0.46060055]]],
 
 
        [[[-0.67745703, -0.7783843 , -0.69385177, -0.42587894,
           -0.44143984, -0.3489665 , -0.68408704, -0.7860846 ,
           -0.55942756, -0.49254662]

In [9]:
new_model.optimizer()

TypeError: 'Adam' object is not callable